In [1]:
import os 
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import linear_model
import sklearn.metrics as sklm
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20.0, 10.0)
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

In [2]:
dir_path = os.path.dirname(os.path.realpath('__file__'))
print(dir_path)

/root/Machine-Learning-Study-Groups


Read the dataset [ features ]

In [3]:
advworkscusts = pd.read_csv(dir_path + '/Datasets/Regression/AdvWorksCusts.csv')
print(advworkscusts.shape)
advworkscusts.head()

(16519, 23)


,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,1966-04-08,Bachelors,Professional,M,M,1,0,0,2,137947
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,1965-05-14,Bachelors,Professional,M,S,0,1,3,3,101141
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,1965-08-12,Bachelors,Professional,M,M,1,1,3,3,91945
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,1968-02-15,Bachelors,Professional,F,S,0,1,0,0,86688
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,1968-08-08,Bachelors,Professional,F,S,1,4,5,5,92771


Read the dataset [ Labels ]

In [4]:
spend = pd.read_csv(dir_path + '/Datasets/Regression/AW_AveMonthSpend.csv')
print(spend.shape)
spend.head()

(16519, 2)


,CustomerID,AveMonthSpend
0,11000,89
1,11001,117
2,11002,123
3,11003,50
4,11004,95


drop duplicates row

In [5]:
advworkscusts.drop_duplicates(subset = 'CustomerID', keep = 'first', inplace = True)
print(advworkscusts.shape)
print(advworkscusts.CustomerID.unique().shape)


(16404, 23)
(16404,)


Here we merge the label column that name (AveMonthSpend) to the features dataset to be all thing in ne dataset or one table.

In [6]:
customerData = pd.merge(advworkscusts, spend)
print(customerData.shape)
customerData.head()


(16519, 24)


,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,Bachelors,Professional,M,M,1,0,0,2,137947,89
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,Bachelors,Professional,M,S,0,1,3,3,101141,117
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,Bachelors,Professional,M,M,1,1,3,3,91945,123
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,Bachelors,Professional,F,S,0,1,0,0,86688,50
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,Bachelors,Professional,F,S,1,4,5,5,92771,95


In this part, we convert BirthDate to age add assign it to his column.

In [7]:
from datetime import date

def calculate_age(bornyear, bornmonth, bornday):
    today = date.today()
    return today.year - bornyear - ((today.month, today.day) < (bornmonth, bornday))

In [8]:
t = []

for i in customerData['BirthDate']:
  birthday = str(i)
  birthday = birthday.split('-')
  b = calculate_age(int(birthday[0]), int(birthday[1]), int(birthday[2]))
  t.append(b)
  
customerData['BirthDate'] = t

We can see the edit now on BirthDate column.

In [9]:
 customerData['BirthDate']

0        52
1        53
2        53
3        50
4        50
5        53
6        53
7        54
8        54
9        54
10       54
11       55
12       50
13       50
14       50
15       39
16       39
17       40
18       40
19       40
20       40
21       72
22       72
23       72
24       72
25       71
26       71
27       71
28       71
29       70
         ..
16489    76
16490    39
16491    40
16492    40
16493    40
16494    40
16495    41
16496    41
16497    41
16498    39
16499    40
16500    41
16501    41
16502    75
16503    57
16504    57
16505    57
16506    58
16507    58
16508    58
16509    58
16510    59
16511    59
16512    59
16513    59
16514    59
16515    60
16516    58
16517    58
16518    59
Name: BirthDate, Length: 16519, dtype: int64

In [10]:
gender = {'M': 0,'F': 1} 

Encode the string features to numerical features

In [11]:
customerData.Gender = [gender[item] for item in customerData.Gender] 

In [12]:
matr  = {'M': 0,'S': 1} 
customerData.MaritalStatus = [matr[item] for item in customerData.MaritalStatus] 
customerData.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,Bachelors,Professional,0,0,1,0,0,2,137947,89
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,Bachelors,Professional,0,1,0,1,3,3,101141,117
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,Bachelors,Professional,0,0,1,1,3,3,91945,123
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,Bachelors,Professional,1,1,0,1,0,0,86688,50
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,Bachelors,Professional,1,1,1,4,5,5,92771,95


In [13]:
# Create x, where x the 'scores' column's values as floats
x = customerData[['AveMonthSpend']].values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)

# Run the normalizer on the dataframe
customerData['AveMonthSpend'] = pd.DataFrame(x_scaled)

# Build Model

In [14]:
# Create x, where x the 'scores' column's values as floats
x = customerData[['YearlyIncome']].values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)

# Run the normalizer on the dataframe
customerData['YearlyIncome'] = pd.DataFrame(x_scaled)


In [15]:
# Create x, where x the 'scores' column's values as floats
x = customerData[['BirthDate']].values.astype(float)

# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)

# Run the normalizer on the dataframe
customerData['BirthDate'] = pd.DataFrame(x_scaled)



In [16]:
customerData.drop(['CustomerID', 'Title','MiddleName','FirstName','LastName','Suffix','AddressLine1','AddressLine2','City','StateProvinceName','Occupation','Education','CountryRegionName','PostalCode','PhoneNumber'], axis=1)



,BirthDate,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend
0,0.211268,0,0,1,0,0,2,0.686872,0.435065
1,0.225352,0,1,0,1,3,3,0.490079,0.616883
2,0.225352,0,0,1,1,3,3,0.440910,0.655844
3,0.183099,1,1,0,1,0,0,0.412802,0.181818
4,0.183099,1,1,1,4,5,5,0.445327,0.474026
5,0.225352,0,1,1,1,0,0,0.501083,0.363636
6,0.225352,1,1,1,1,0,0,0.402472,0.207792
7,0.239437,0,0,1,2,3,3,0.536157,0.701299
8,0.239437,1,1,1,3,4,4,0.419844,0.409091
9,0.239437,0,1,0,1,0,0,0.518192,0.337662


In [17]:
BirthDate = customerData['BirthDate'].values
Gender = customerData['Gender'].values
HomeOwnerFlag = customerData['HomeOwnerFlag'].values
NumberCarsOwned = customerData['NumberCarsOwned'].values
NumberChildrenAtHome = customerData['NumberChildrenAtHome'].values
TotalChildren = customerData['TotalChildren'].values
YearlyIncome = customerData['YearlyIncome'].values
AveMonthSpend = customerData['AveMonthSpend'].values

m = len(BirthDate)
x0 = np.ones(m)
X = np.array([x0, BirthDate, HomeOwnerFlag,Gender,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome]).T
# Initial Coefficients
B = np.array([0, 0, 0, 0, 0, 0, 0, 0])
Y = np.array(AveMonthSpend)
alpha = 0.0001


In [18]:
def cost_function(X, Y, B):
    m = len(Y)
    J = np.sum((X.dot(B) - Y) ** 2)/(2 * m)
    return J


In [19]:
inital_cost = cost_function(X, Y, B)
print(inital_cost)


0.06926140269680922


In [20]:
def gradient_descent(X, Y, B, alpha, iterations):
    cost_history = [0] * iterations
    m = len(Y)
    
    for iteration in range(iterations):
        # Hypothesis Values
        h = X.dot(B)
        # Difference b/w Hypothesis and Actual Y
        loss = h - Y
        # Gradient Calculation
        gradient = X.T.dot(loss) / m
        # Changing Values of B using Gradient
        B = B - alpha * gradient
        # New Cost Value
        cost = cost_function(X, Y, B)
        cost_history[iteration] = cost
        
    return B, cost_history

In [21]:


# 100000 Iterations
newB, cost_history = gradient_descent(X, Y, B, alpha, 100000)

# New Values of B
print(newB)

# Final Cost of new B
print(cost_history[-1])



[ 0.21190446  0.02012483  0.05136993 -0.1545391   0.01294907  0.07390559
  0.00219229  0.12865483]
0.0017756275499425424


In [22]:

def rmse(Y, Y_pred):
    rmse = np.sqrt(sum((Y - Y_pred) ** 2) / len(Y))
    return rmse

# Model Evaluation - R2 Score
def r2_score(Y, Y_pred):
    mean_y = np.mean(Y)
    ss_tot = sum((Y - mean_y) ** 2)
    ss_res = sum((Y - Y_pred) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    return r2

Y_pred = X.dot(newB)

print(rmse(Y, Y_pred))
print(r2_score(Y, Y_pred))

0.05959240807254791
0.8868668464534567
